[View in Colaboratory](https://colab.research.google.com/github/jsedoc/Sharvin-Shah-Spring-2018/blob/master/tree2seq_colab.ipynb)

In [4]:
import sys
print(sys.version_info)

!pip install http://download.pytorch.org/whl/cu90/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl

sys.version_info(major=3, minor=6, micro=3, releaselevel='final', serial=0)
    86% |███████████████████████████▊    | 549.4MB 40.1MB/s eta 0:00:03

    100% |████████████████████████████████| 633.1MB 42.7MB/s 
  Found existing installation: torch 0.4.0
    Uninstalling torch-0.4.0:
      Successfully uninstalled torch-0.4.0
